In [ ]:
import torch
def expand(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'], st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'], st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

def expand_truncate(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'], st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'], st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.startswith('classifier.') or k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

def expand_three_times_truncate(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'],
         st_dict['layoutlmv3.embeddings.position_ids'],
         st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'],
         st_dict['layoutlmv3.embeddings.position_embeddings.weight'],
         st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    d = ['x', 'y', 'w', 'h']
    for dd in d: 
        st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'] = torch.cat(
            (st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'], 
             st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'])
            , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    print("expand x_position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.x_position_embeddings.weight'].shape)
    print("expand y_position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.y_position_embeddings.weight'].shape)
    print("expand w_position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.w_position_embeddings.weight'].shape)
    print("expand h_position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.h_position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.startswith('classifier.') or k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

In [ ]:
from layoutlmft.models.layoutlmv3.modeling_layoutlmv3 import LayoutLMv3ForRelationExtraction, LayoutLMv3ForTokenClassification
# model_ft = LayoutLMv3ForTokenClassification.from_pretrained(
#     "HYPJUDY/layoutlmv3-large-finetuned-funsd"
# )

In [ ]:
# re_pretrain = LayoutLMv3ForRelationExtraction.from_pretrained(
#     "microsoft/layoutlmv3-large"
# )
ner_pretrain = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-large"
)

In [ ]:
ner_st_dict = ner_pretrain.state_dict()
expand_truncate(ner_st_dict, "layoutlmv3-large-ner-1028/pytorch_model.bin")

In [ ]:
ner_st_dict = ner_pretrain.state_dict()
expand_three_times_truncate(ner_st_dict, "layoutlmv3-large-ner-1542/pytorch_model.bin")

In [ ]:
from layoutlmft.models.layoutlmv3.modeling_layoutlmv3 import LayoutLMv3ForTokenClassification
model_from_path = LayoutLMv3ForTokenClassification.from_pretrained(
    "layoutlmv3-base-2048",
    # num_labels=2
)


In [ ]:
for k, v in model_from_path.state_dict().items():
    print(k, v.shape)

In [5]:
import torch
torch.cuda.is_available()

True